In [1]:
!pip install tensorflow
!pip install keras

In [2]:
import collections
import helper
import numpy as np
import tensorflow as tf
import re

from numpy import array
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from numpy import argmax
from keras.models import load_model
from keras.layers import Dropout
 
from keras.optimizers import gradient_descent_v2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.losses import sparse_categorical_crossentropy

In [3]:
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [4]:
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [5]:
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

In [6]:
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [24]:
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
  model = Sequential()
  model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
  model.add(Dropout(0.5))
  model.add(LSTM(n_units))
  model.add(Dropout(0.3))
  model.add(RepeatVector(tar_timesteps))
  model.add(LSTM(n_units, return_sequences=True))
  model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
 
  return model

In [8]:
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [9]:
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [10]:
def evaluate_model(model, tokenizer, sources):
	predicted = list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		#raw_target, raw_src = raw_dataset[i]
		print('predicted=[%s]' % (translation))
		#actual.append([raw_target.split()])
		predicted.append(translation.split())
        

In [11]:
listofLines=[]

file1 = open('/content/sentencesTrain.txt', 'r',encoding='cp1252')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    # if count==100:
    #    break
    line1 = line.strip() 
    cleanString = re.sub('\W+',' ', line1)#or some other preprocessing
    line1 = cleanString.strip() 
    listofLines.append(line1) 
    #print(len(line1))
    #print("Line{}: {}".format(count, line.strip()))
print(listofLines)
print(len(listofLines))

['When does the party start at 8 or 9', 'When my sister goes water tubing I will have to watch her and make sure to stop the boat when she falls off', 'If mom has time she will bring cookies', 'No I wasn t old enough to vote at that time', 'My friend wants to buy a plaid shirt', 'If it snows tomorrow then it s a guarantee that school will be closed', 'I just drank it and it tastes fine', 'If mom doesn t walk to the store we won t have milk', 'Mother will give me twenty dollars if I clean the kitchen', 'Who listened to the teacher', 'I just read a book', 'It can be expressed through speech or through sign language', 'It doesn t matter to me I like anything', 'No today s Thursday which means tomorrow s Friday', 'Where did the friends drive to', 'My friend is playing video games', 'What does my sister want for her birthday', 'Has your friend experienced travelling', 'Bob hasn t sent the letter', 'If my family decides to vacation every year they will go to Puerto Rico', 'The DVD that Netfl

In [12]:
listoftokens=[]

file1 = open('/content/tokensTrain.txt', 'r',encoding='cp1252')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    # if count==100:
    #    break
    line1 = line.strip() #or some other preprocessing
    listoftokens.append(line1) 
    #print(line.strip())
    #print("Line{}: {}".format(count, line.strip()))
print(listoftokens)
print(len(listoftokens))

['(P)PARTY,START,WHICH,TIME,EIGHT,TIME,NINE,(P)PARTY,START,WHICH,TIME', '(G/Q)POSS-1p,SISTER,(2h)IX-3p:i,tube skipping on water,IX-1p,GO-BY-BOAT,IX-1p,LOOK,MAKE,REALLY,person flips off,IX-1p,STOP,SISTER,(2h)IX-3p:i,tube skipping on water,GO-BY-BOAT,MAKE,STOP', 'MOTHER,IX-loc:i,#IF+,(1h)HAVE,TIME+,IX-3p:i,FUTURE,BRING,COOKIE,TIME+,BRING,COOKIE', '#NO+,IX-1p,NOT,OLD,ENOUGH,TO/UNTIL,VOTE,THAT,TIME,part:indef,ENOUGH,TO/UNTIL,VOTE,TIME,part:indef', 'FRIEND,WANT_2,BUY,PLAID,(2h)SHIRT,FRIEND,WANT_2,BUY,(2h)SHIRT', '#IF,TOMORROW,SNOW,GUARANTEE,SCHOOL,FUTURE,CLOSE-DOOR,SNOW,GUARANTEE,SCHOOL,CLOSE-DOOR', 'IX-1p,RECENT-PAST,DRINK,TASTE,FINE++,(1h)part:indef', '#IF,MOTHERwg,IX-3p:i,NOT,person walking,SELL,IX-1p-pl-arc,REFUSE,(1h)HAVE,MILK,part:indef,SELL,(1h)part:indef,part:indef', 'FINEwg,MOTHERwg,FUTURE,GIVE-1p,TWENTY,DOLLAR,#IF,IX-1p,NICE/CLEAN,(K)KITCHEN,DOLLAR,NICE/CLEAN,(K)KITCHEN', 'TEACH+AGENT,IX-loc:i,IX-3p-pl-arc,HEAR/LISTEN,IX-3p:i,WHO,TEACH,AGENT,BUOY,IX-3p:j', 'IX-1p,RECENT-PAST,READ+

In [13]:
listofPreds=[]

file1 = open('/content/sentencesTest.txt', 'r',encoding='cp1252')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    # if count==10:
    #    break
    line1 = line.strip() 
    cleanString = re.sub('\W+',' ', line1)#or some other preprocessing
    line1 = cleanString.strip() 
    listofPreds.append(line1) 
    #print(len(line1))
    #print("Line{}: {}".format(count, line.strip()))
print(listofPreds)
print(len(listofPreds))

['Do you like Boston', 'My friend likes to VP rather than text because he likes to communicate in ASL', 'When Bob enters the surprise party everyone will spray silly string at him', 'If your friend gets fucked up I won t take care of him', 'When a bad storm comes to Boston I will stay home', 'If all the students pass the test then there will be no school tomorrow', 'Have you ever lived through a winter in Boston It snows a lot', 'If my friends are traveling I have to join them', 'If Mother does not clean the kitchen I have to do it myself', 'I invited my friend to a BBQ because it s Fourth of July soon', 'If a man has a wife then he loves her', 'My friends are planning a surprise event', 'Bob hasn t sent the letter yet', 'No I wouldn t stop vomiting if I had pineapple cream cheese Lox only is fine with me', 'I didn t enjoy talking to Donna and Jessica', 'When a bad storm comes to Boston I have to stay home', 'If I pay rent now I will have to cancel my vacation', 'Yes I do have homework

In [14]:
eng_tokenizer = create_tokenizer(listofLines)
#print(eng_tokenizer.word_index)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length=max_length(listofLines)
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
trainX = encode_sequences(eng_tokenizer, eng_length, listofLines)
print(trainX)

sign_tokenizer = create_tokenizer(listoftokens)
print(sign_tokenizer.word_index)
sign_vocab_size = len(sign_tokenizer.word_index) + 1
sign_length=max_length(listoftokens)
print('English Vocabulary Size: %d' % sign_vocab_size)
print('English Max Length: %d' % (sign_length))
trainY = encode_sequences(sign_tokenizer, sign_length, listoftokens)
print(trainY)
trainY = encode_output(trainY, sign_vocab_size)
print(trainY)

English Vocabulary Size: 1076
English Max Length: 31
[[  20   41    1 ...    0    0    0]
 [  20    4   71 ...    0    0    0]
 [   5  136   47 ...    0    0    0]
 ...
 [  20    4  127 ...    0    0    0]
 [   5   96 1074 ...    0    0    0]
 [   1  103   19 ...    0    0    0]]
{'ix': 1, '1p': 2, 'i': 3, '3p': 4, 'part': 5, 'indef': 6, 'fs': 7, '1h': 8, 'friend': 9, '2h': 10, '2': 11, 'not': 12, 'loc': 13, 'poss': 14, 'go': 15, 'if': 16, 'out': 17, 'work': 18, 'why': 19, 'no': 20, '2p': 21, 'who': 22, 'in': 23, 'p': 24, 'future': 25, 'want': 26, 'party': 27, 'group': 28, 'together': 29, 'nothing': 30, 'finish': 31, 'arrive': 32, 'up': 33, 'car': 34, '25': 35, 'when': 36, 'night': 37, 'none': 38, 'pl': 39, 'j': 40, 'have': 41, 'buy': 42, 'start': 43, 'do': 44, 'wave': 45, 'arc': 46, 'game': 47, 'now': 48, 'ns': 49, 'make': 50, 'play': 51, 'agent': 52, 'must': 53, 'time': 54, 'past': 55, 'really': 56, 'school': 57, 'neg': 58, 'look': 59, 'teach': 60, '1': 61, 'proceed': 62, 'wow': 63, 

In [25]:
model = define_model(eng_vocab_size,sign_vocab_size, eng_length, sign_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=300, batch_size=64, validation_split=0.3, callbacks=[checkpoint])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 31, 256)           275456    
                                                                 
 dropout (Dropout)           (None, 31, 256)           0         
                                                                 
 lstm_6 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 repeat_vector_3 (RepeatVect  (None, 13, 256)          0         
 or)                                                             
                                                                 
 lstm_7 (LSTM)               (None, 13, 256)           525312    
                                                      

In [26]:
testX = encode_sequences(eng_tokenizer, eng_length, listofPreds)
model = load_model('model.h5')
#evaluate_model(model, eng_tokenizer, trainX)

In [27]:
predicted = list()
for i, source in enumerate(trainX):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, sign_tokenizer, source)
		#raw_target, raw_src = raw_dataset[i]
		print('predicted=[%s]' % (translation))
		#actual.append([raw_target.split()])
		predicted.append(translation)
print(predicted)

predicted=[ix ix ix ix ix indef indef]
predicted=[ix ix ix ix ix ix ix ix 1p 1p 1p 1p 1p]
predicted=[ix ix ix ix ix ix ix ix indef indef indef indef indef]
predicted=[ix ix ix ix ix ix indef indef indef indef indef indef indef]
predicted=[ix ix ix ix ix ix indef indef indef indef indef indef indef]
predicted=[ix ix ix ix ix ix ix ix indef indef indef indef indef]
predicted=[ix ix ix]
predicted=[ix ix ix ix ix ix ix ix i i i indef indef]
predicted=[ix ix ix ix ix ix ix ix i i indef indef indef]
predicted=[ix ix ix ix indef]
predicted=[ix ix ix]
predicted=[ix ix ix ix ix ix ix ix i indef indef indef indef]
predicted=[ix ix ix ix ix ix ix ix indef indef indef indef indef]
predicted=[ix ix ix]
predicted=[ix ix ix ix]
predicted=[ix ix ix ix ix indef indef]
predicted=[ix ix ix ix ix ix indef indef indef indef indef indef indef]
predicted=[ix ix ix]
predicted=[ix ix ix ix ix indef indef indef indef]
predicted=[ix ix ix ix ix ix ix ix i i i indef indef]
predicted=[ix ix ix ix ix ix ix ix indef

In [19]:
textfile = open("tokensTest.txt", "w")
for element in predicted:
    textfile.write(element + "\n")
textfile.close()

In [34]:
listofPred=[]

file1 = open('/content/tokensTest.txt', 'r',encoding='cp1252')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    # if count==10:
    #    break
    line1 = line.strip() 
    words = line1.split(" ")
    joined_string = ",".join(words)
    print(joined_string)
    #cleanString = re.sub('\W+',' ', line1)#or some other preprocessing
    #line1 = cleanString.strip() 
    listofPred.append(joined_string) 
    #print(len(line1))
    #print("Line{}: {}".format(count, line.strip()))
print(listofPred)
print(len(listofPred))

time,eight,time,nine,p,party,party,time,time
tube,skipping,on,water,go,by,boat,stop,stop
ix,3p,i,future,bring,bring,time,cookie,cookie
part,indef,enough,to,until,vote,time,part,indef
plaid,2h,shirt,friend,want,2,2h,2h,2h
school,future,close,door,snow,guarantee,school,close,close
1p,recent,past,drink,taste,fine,part,part,indef
milk,part,indef,sell,1h,part,part,indef,indef
nice,clean,k,kitchen,dollar,nice,nice,kitchen,kitchen
3p,i,who,teach,agent,buoy,ix,3p,j
1p,recent,past,read,book,ix,1p,1p,book
2,2h,talkwg,minor,express,sign,current,current,road
2h,thing,2h,any,2h,thing,i,don't,know
wave,no,now,thursday,mean,tomorrow,mean,now,mean
friend,group,together,together,drive,friend,drive,drive,drive
i,play,video,game,friend,now,play,game,game
birthday,1h,do,do,1h,part,indef,sister,want
friend,finish,experience,trip,qmwg,friend
i,not,yet,send,letter,letter,mail,mail,mail
year,depart,ns,pr,family,decide,vacation,year,year
1h,scrape,1h,part,indef,send,recieve,dvd,dvd
jack,fs,sue,meet,past,finish

In [35]:
textfile = open("tokensTest.txt", "w")
for element in listofPred:
    textfile.write(element + "\n")
textfile.close()